In [1]:
import pandas as pd
import numpy as np
from collections import Iterable


In [4]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


KeyError: 'CLOUDSDK_CONFIG'

In [ ]:
tcedf = pd.read_csv('/content/drive/My Drive/TCE-DAT_historic-exposure_1950-2015.csv',skiprows = 6)
tcedf = tcedf[tcedf['year'] >= 1980]

In [ ]:
emdatdf = pd.read_csv('/content/drive/My Drive/emdat_cleaned.csv')
emdatdf = emdatdf[emdatdf['Year'] >= 1980 ]

In [ ]:
tcedf.shape, emdatdf.shape

In [ ]:
tcedf.head(1)

In [ ]:
emdatdf.head(1)

In [ ]:
### emdat processing

In [ ]:
### emdat Try dropping rows without storm names. 
emdat_noname_df = emdatdf.dropna(subset=['Event name'])

In [ ]:
### Helper function - Flatten nested list
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [ ]:
emdat_cols_drop = ['Latitude','Longitude','Magnitude (scale)',
                   'EU member','OFDA response', 'Appeal',
                  'Declaration', "Aid contribution ('000$)",
                  'Associated disater', 'Associated disater 2']

emdat_noname_df = emdat_noname_df.drop(columns=emdat_cols_drop)

## Some event names have year numbers. Strip this out.
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace('\d+', '')

In [ ]:
### Remove words like "Hurricane", "Cyclone" e.t.c from names column
import re

to_remove = ['Hurricane', 'Tropical', 'Cyclone','depression', 'storm', 'cylone',
             'Storm', r"\(.*\)", "'", '"', "Typhoon", 'CYCLONE', 'cyclone', 'Tropcal', 
             'TYphoon', 'Topical', 'Depression', 'Typhhon', 'Tyhoon', 'Cclone', 
             'TRopical', 'STROM', 'strom']


for each in to_remove:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace(each, '')


'''
for each in to_remove:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.replace(each, ''))
'''

In [ ]:
split_terms = ["&", " AND ", "/", " ET ", ","]

for x in split_terms:
  emdat_noname_df['Event name'] = emdat_noname_df['Event name'].str.replace(x, ' & ')


## Indexes of entries with joined names
indexes = emdat_noname_df[emdat_noname_df['Event name'].str.contains("|".join(split_terms))].index


subset_emdat = emdat_noname_df.copy().loc[indexes]
subset_emdat['Event name'] = subset_emdat['Event name'].str.split('&')


cols = subset_emdat.columns.values
cols = [v for v in cols if v != 'Event name']



temp_emdat = []

for _, row in subset_emdat.iterrows():
  for d in row['Event name']:
    temp_emdat.append(flatten([list(row[cols]), d]))


ext_subset_emdat = pd.DataFrame(temp_emdat, columns=cols + ['Event name'])


In [ ]:
emdat_noname_df.loc[~emdat_noname_df.index.isin(indexes)].shape

In [ ]:
emdat_noname_df.shape

In [ ]:
emdat_noname_df.head(1)

In [ ]:
ext_subset_emdat.head(1)

In [ ]:
### Order of columns not the same. Enforce uniformity
emdat_noname_df = pd.concat([emdat_noname_df.loc[~emdat_noname_df.index.isin(indexes)], 
                             ext_subset_emdat[emdat_noname_df.columns.values]], sort=True)

In [ ]:
emdat_noname_df['Event name'].unique()

In [ ]:
## Remove rows without event name
emdat_noname_df[emdat_noname_df['Event name'] != ""]

## Strip leading spaces .str.strip()
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.strip())

In [ ]:
### It appears some observations have name as simply "Hurricane" 
## and removing these leaves the name empty. This should be dropped.
mask = emdatdf['Event name'].str.contains(r'\b(?:{})\b'.format('|'.join(to_remove)))
mask = mask.replace(np.nan, False)


## Convert names to uppercase
emdat_noname_df['Event name'] = emdat_noname_df['Event name'].apply(lambda x: x.upper())

In [ ]:
emdat_noname_df['Event name'].value_counts()

In [ ]:
emdat_noname_df.shape

In [ ]:
#### Ibtracs

In [ ]:
ibtracdf = pd.read_csv('/content/drive/My Drive/IBTrACS-ALL-list-v04r00-lines-dbf.csv')
ibtracdf = ibtracdf[ibtracdf['year'] >= 1980]
tup_zipped = list(set(tuple(zip(ibtracdf.SID, ibtracdf.NAME))))

## Create a dictionary with ibtracs SID & storm name
SID_Namedict = {x:y for x, y in tup_zipped }

In [ ]:
###Tce dat processing

In [ ]:
tcedf_copy = tcedf.copy()

## Create column "name" using ibtracID
tcedf_copy['name'] = tcedf_copy['IBTrACS_ID'].apply(lambda x: SID_Namedict.get(str(x)))

## Drop rows without event name
tcedf_copy = tcedf_copy[tcedf_copy['name'] != "NOT_NAMED"]

# Drop rows with no names on both name & tc_name
tcedf_copy = tcedf_copy[(tcedf_copy['TC_name'] != "UNNAMED") & (tcedf_copy['name'].notna())]

In [ ]:
tcedf_copy.head()

In [ ]:
## SINA appears twice. 1990 & 1980 We would
# need to merge on both name and year. 
tcedf_copy[tcedf_copy['TC_name'] == "SINA"]

In [ ]:
## TC_NAME
tcedf_copy["TC_name"][tcedf_copy["TC_name"] == 'BELLY'] = "BETTY"
tcedf_copy["TC_name"][tcedf_copy["TC_name"] == 'BRENDA'] = "BRENDAN"




id_split = ['1980073S09133', '1987035S12160', '1988285N09318', '1989209N22130', 
            '1996282N11162', '1998036S13135', '2013130N04093']


for eachsplit in id_split:
  tcedf_copy['TC_name'][tcedf_copy['IBTrACS_ID'] == eachsplit] = tcedf_copy['name'].str.split(":").str[0]
  tcedf_copy['name'][tcedf_copy['IBTrACS_ID'] == eachsplit] = tcedf_copy['name'].str.split(":").str[1]

In [ ]:
## Columns to keep
tce_dat_columns = ['year', 'IBTrACS_ID', 'name', 'TC_name', 'countries_affected', 'ISO3', 'v_land_SI',
                   'v_land_kn', '34kn_pop', '34kn_assets', '64kn_pop', '64kn_assets', 
                   '96kn_pop', '96kn_assets']

tcedf_copy = tcedf_copy[tce_dat_columns]                  

In [ ]:
## Entries where names don't match. This is 
# either due to typos or multiple naming

tcedf_copy[["IBTrACS_ID", "TC_name", "name"]][tcedf_copy['TC_name'] != tcedf_copy['name']]

In [ ]:
### Compare emdat & Tce dat

In [ ]:
emdat_list = list(emdat_noname_df['Event name'])

tce_lists = [list(tcedf_copy['TC_name']), list(tcedf_copy['name'])]

for l in tce_lists:
  results = {}
  for i in emdat_list:
    results[i] = l.count(i) 

  count = 0
  for each in results:
    if results[each] != 0:
      count += 1

  print(count) ##478 & 476

In [ ]:
emdatzip = { x:y for x, y in zip(emdat_noname_df['Event name'], emdat_noname_df['Year'])}
tcezip = { x:y for x, y in zip(tcedf_copy['TC_name'], tcedf_copy['year'])}

In [ ]:
tcedf_copy.shape, emdat_noname_df.shape

In [ ]:
#####################
###################
##################
### New Merge Attempt
tcedf_copy.head(2)

In [ ]:
emdat_noname_df.head(2)

In [ ]:
##Inner Join

merged_one = pd.merge(tcedf_copy, emdat_noname_df, left_on=['TC_name', 'year', 'ISO3'], right_on=['Event name', 'Year', 'ISO'])
merged_two = pd.merge(tcedf_copy, emdat_noname_df, left_on=['name', 'year', 'ISO3'], right_on=['Event name', 'Year', 'ISO'])
merged = pd.concat([merged_one, merged_two], sort=True).drop_duplicates()

In [ ]:
merged_one.shape, merged_two.shape, merged.shape

In [ ]:
merged[merged['Total affected'].isna()].shape

In [ ]:
merged[merged['Total affected'].isna()]

In [ ]:
merged[merged['64kn_pop'].isna()].shape

In [ ]:
merged[(merged['Total affected'].isna() & merged['34kn_pop'].isna())].shape

In [ ]:
### Outer Join

##Inner Join

merged_one_outer = pd.merge(tcedf_copy, emdat_noname_df, 
                      left_on=['TC_name', 'year', 'ISO3'], 
                      right_on=['Event name', 'Year', 'ISO'], 
                      how='outer')
merged_two_outer = pd.merge(tcedf_copy, emdat_noname_df, 
                      left_on=['name', 'year', 'ISO3'], 
                      right_on=['Event name', 'Year', 'ISO'], 
                      how='outer')
merged_outer = pd.concat([merged_one_outer, merged_two_outer], sort=True).drop_duplicates()

In [ ]:
merged_outer.shape

In [ ]:
merged_outer.head(2)

In [ ]:
merged_outer[merged_outer['Total affected'].isna()].shape

In [ ]:
merged_outer[(merged_outer['Total affected'].isna() & merged_outer['34kn_pop'].isna())].shape